In [1]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#Youtube Data API Packages: 
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
from googleapiclient.discovery import build
import argparse
"""
from googleapiclient.errors import HttpError"""

#Modeling
#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
#from sklearn.svm import SVC
#from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

#NLP & Text Management
'''from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer'''
import re

#Time analysis
import time
import datetime as dt
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('seaborn-whitegrid')
sns.set_style('whitegrid')

In [2]:
df_main = pd.read_csv('./data/clean_data_nocomments_noviews_02.10.18.csv')

In [22]:
df_target = df_main.loc[:, ['vidId', 'title', 'viewCount', 'publishedAt']]

In [23]:
df_target.shape

(8694, 4)

# Plan:
1. Get trends data
2. For comments, pull from start date

### Generate search strings

Examples:  
https://trends.google.com/trends/explore?date=all&q=cardi%20b%20bodak%20yellow

https://trends.google.com/trends/explore?date=all_2008&gprop=youtube&q=cardi%20b%20bodak%20yellow

In [129]:
def get_http(input_str):
    hyp_split = input_str.split('-')
    try:
        paren_split = hyp_split[1].split('(')
        ret_str = (' '.join([hyp_split[0], paren_split[0]] )).replace('  ', ' ').replace('  ', ' ').lower().replace(' ', '%20')
    except:
        ret_str = input_str.replace('  ', ' ').replace('  ', ' ').lower().replace(' ', '%20')

    if ret_str[len(ret_str)-3:] == '%20':
        ret_str=ret_str[:len(ret_str)-3]
        
    ret_str = 'https://trends.google.com/trends/explore?date=2008-01-01%202018-10-03&gprop=youtube&q='+ret_str
    return ret_str

In [102]:
test_string = 'Some Random person- Tonight is  the night (Official Music) ft some pitbull, probably'
temp(test_string)

In [131]:
df_target['search_term'] = df_target.title.apply(lambda x: get_http(x))

In [134]:
df_target.search_term[100]

'https://trends.google.com/trends/explore?date=2008-01-01%202018-10-03&gprop=youtube&q=machine%20gun%20kelly%20habits'

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [0       15
1       13
2       -1
3       14
4       -1
5       12
6       18
7       12
8       13
9        7
10      19
11      23
12       5
13      10
14      19
15      -1
16       4
17      12
18      18
19      26
20      -1
21       9
22      14
23      13
24       8
25       4
26      15
27      13
28      -1
29      -1
        ..
8664    11
8665    11
8666     9
8667    27
8668    10
8669    11
8670     8
8671    12
8672    11
8673     9
8674    -1
8675    11
8676    10
8677    11
8678    25
8679    10
8680    -1
8681    14
8682    15
8683    12
8684     6
8685    13
8686    10
8687     8
8688    18
8689    10
8690     7
8691     7
8692    12
8693    -1
Name: title, Length: 8694, dtype: int64] of <class 'pandas.core.series.Series'>

### Pull HTML

In [156]:
from bs4 import BeautifulSoup

#This approach does not work -- google has some other authorization requirements
#see - https://mathewanalytics.com/2017/01/30/extract-google-trends-data-with-python-2/
"""# Target web page:
url = df_target.search_term[100]
# Establishing the connection to the web page:
response = requests.get(url, headers={'User-agent': 'CBH_plz_love_me'})
# Pull the HTML string out of requests and convert it to a Python string.
html = response.text
soup = BeautifulSoup(html, 'lxml')

response

soup

soup.findAll('a')"""

"# Target web page:\nurl = df_target.search_term[100]\n# Establishing the connection to the web page:\nresponse = requests.get(url, headers={'User-agent': 'CBH_plz_love_me'})\n# Pull the HTML string out of requests and convert it to a Python string.\nhtml = response.text\nsoup = BeautifulSoup(html, 'lxml')\n\nresponse\n\nsoup\n\nsoup.findAll('a')"

In [157]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")


In [158]:
url = df_target.search_term[100]

In [159]:
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')

# Planning

In [ ]:
#Gonna find out how many of these we can do...

Hang on - actually we should start the comments from the beginning anyway. If can't find it, can't find it. 

Sub - task: identify classical music by No. ## format, then use key terms (high enough frequency) to find these later. 
Value add: basics of building own text processing features, classification challenge AND you can use the general downward trend in classical music to further specify the shape of the views.
